Re Ranking

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser 

/Users/maruthienugula/RAG_learnings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load a text file
loader = TextLoader("langchain-sample.txt")
raw_docs = loader.load()

In [3]:
# split raw docs into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs = splitter.split_documents(raw_docs)
docs

[Document(metadata={'source': 'langchain-sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain-sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain-sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [15]:
query = "How can i use langchain to build an application with memory and tools?"

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectorstore = FAISS.from_documents(docs,embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={'k':8})

In [7]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x128550a40>, search_kwargs={'k': 8})

In [8]:
# prompt and use the llm
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm=init_chat_model("groq:llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x127fbe3c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x139641d30>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [17]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

User Question: "{question}"

Documents:
{documents}

Instructions:
- When replying give the output first then all relevant content
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [18]:
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='d4860aab-e49c-40ee-a2b5-ca02b9724262', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='7f56a373-e3f6-4ab0-8494-224183a7b784', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(id='7a52ea98-9a38-492f-99f1-0c2be1e7ad32', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond mo

In [19]:
chain = prompt | llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\nUser Question: "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- When replying give the output first then all relevant content\n- Think about the relevance of each document to the user\'s question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n')
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x127fbe3c0>, async_client=<groq.resources.chat.completions.Asy

In [20]:
doc_lines = [f"{i+1}.{doc.page_content} with metadata : {doc.metadata}" for i,doc in enumerate(retrieved_docs)]
doc_lines

["1.LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents. with metadata : {'source': 'langchain-sample.txt'}",
 "2.LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation. with metadata : {'source': 'langchain-sample.txt'}",
 "3.LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful. with metadata : {'source': 'langchain-sampl

In [26]:
formatted_docs = "\n".join(doc_lines)
formatted_docs

"1.LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents. with metadata : {'source': 'langchain-sample.txt'}\n2.LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation. with metadata : {'source': 'langchain-sample.txt'}\n3.LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful. with metadata : {'source': 'langchain-sample.txt'}

In [27]:
response = chain.invoke({"question":query,"documents":formatted_docs})
response

'3,1,4,2,6,5\n\nThe ranking is based on the relevance of each document to the user\'s question about using LangChain to build an application with memory and tools. \n\n1. Document 3 is the most relevant because it explicitly mentions "Memory in LangChain" and its role in enabling context retention across multiple steps in a conversation or task, as well as tool integration.\n2. Document 1 is the second most relevant as it provides a general overview of LangChain, including its components for prompt management, chains, memory, and agents, which are all relevant to building an application.\n3. Document 4 is also relevant because it mentions "Memory" and "Agents" in LangChain, which can be used to decide which tools to use and in what order, making it suitable for multi-step tasks.\n4. Document 2 is somewhat relevant as it talks about LangChain\'s integration with third-party services, but it doesn\'t directly address memory and tools.\n5. Document 6 is less relevant as it discusses Retri

In [29]:
indices = [int(x.strip())-1 for x in response.split(",") if x.strip().isdigit()]
indices

[2, 0, 3, 1, 5]

In [30]:
retrieved_docs

[Document(id='d4860aab-e49c-40ee-a2b5-ca02b9724262', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='7f56a373-e3f6-4ab0-8494-224183a7b784', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(id='7a52ea98-9a38-492f-99f1-0c2be1e7ad32', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond mo

In [31]:
reranked_docs = [retrieved_docs[i] for i in indices if 0<=i<=len(retrieved_docs)]
reranked_docs

[Document(id='7a52ea98-9a38-492f-99f1-0c2be1e7ad32', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='d4860aab-e49c-40ee-a2b5-ca02b9724262', metadata={'source': 'langchain-sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='1654f286-c1ac-4fd8-b6b1-f8f1d88fc9b5', metadata={'source': 'langchain-sample.txt'}, page_content='FAISS is a popular library used for fast approximate nearest neighbor search 

In [32]:
# Show results
print("Final re-ranked results")
for i,doc in enumerate(reranked_docs):
    print(f"\nRank {i+1}. {doc.page_content}")

Final re-ranked results

Rank 1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 3. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with